# Generate features (acoustic, phonetic, and lexical) for stimuli
## 0. Some basic functions
### 0.1. Test multicollinearity

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
lme_dir = 'D:\\bsliang_Coganlabcode\\coganlab_ieeg\\projects\\lme\\data'

def check_multicollinearity(X_i,corr_labels):
    
    # Ensure the input is a NumPy array
    if not isinstance(X_i, np.ndarray):
        raise ValueError("Input X_i must be a NumPy array")

    # Compute VIF for each feature
    vif_data = pd.DataFrame()
    vif_data["Feature"] = corr_labels
    vif_data["VIF"] = [variance_inflation_factor(X_i, i) for i in range(X_i.shape[1])]

    # Plot correlation matrix as a heatmap
    plt.figure(figsize=(10, 8))
    correlation_matrix = np.corrcoef(X_i, rowvar=False)  # Compute correlation matrix
    sns.heatmap(correlation_matrix, annot=True, fmt=".1f", cmap="coolwarm", xticklabels=corr_labels, yticklabels=corr_labels,annot_kws={"size": 5})
    plt.title("Feature Correlation Matrix")
    plt.show()

    return vif_data

### 0.2. PCA


In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.decomposition import PCA

def perform_pca_on_acoustic_features(envelope_feature_dict, original_feature_names,n_components=None, visualize=True, min_variance_retention=0.95):
    """
    Performs Principal Component Analysis (PCA) on a dictionary of acoustic features.

    Args:
        envelope_feature_dict (dict): A dictionary where keys are sample/word identifiers
                                      (e.g., 'bacon', 'bagel') and values are lists
                                      of their corresponding acoustic features (e.g., power envelope means
                                      across different frequency bands).
        n_components (int or float or None, optional): The number of components to keep after PCA.
            - If int: The exact number of principal components to return.
            - If float (0.0 < n_components < 1.0): The proportion of variance to explain.
              PCA will select the minimum number of components such that the explained variance
              is greater than or equal to this value.
            - If None (default): All components will be kept. The function will then
              suggest an optimal number based on `min_variance_retention`.
        visualize (bool, optional): If True, generate plots for explained variance and
                                     (if applicable) a scatter plot of the first two principal components.
                                     Defaults to True.
        min_variance_retention (float, optional): Only used if `n_components` is None.
            Specifies the minimum cumulative explained variance ratio (between 0 and 1)
            to retain for suggesting the number of components. Defaults to 0.95.

    Returns:
        tuple: A tuple containing:
            - pca_transformed_df (pd.DataFrame): DataFrame with the transformed principal components.
                                                Rows are samples, columns are principal components (PC1, PC2, ...).
            - pca_model (sklearn.decomposition.PCA): The fitted PCA model object.
            - scaler_model (sklearn.preprocessing.StandardScaler): The fitted StandardScaler model object.
    """

    print("--- Starting PCA Analysis ---")

    # 1. Convert the dictionary to a Pandas DataFrame
    # Each row will represent a sample (word), and each column will represent a feature (frequency band).
    df = pd.DataFrame.from_dict(envelope_feature_dict, orient='index')

    print(f"\nOriginal data shape: {df.shape} (samples x features)")
    # print("Original data (first 5 rows):\n", df.head()) # Uncomment for full data view

    # 2. Data Standardization
    # PCA is sensitive to feature scales, so standardization is a crucial step.
    # StandardScaler transforms data to have a mean of 0 and a standard deviation of 1.
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df)
    scaled_df = pd.DataFrame(scaled_data, index=df.index, columns=df.columns)
    # print("\nScaled data (first 5 rows):\n", scaled_df.head()) # Uncomment for full data view

    # 3. Perform PCA (initial fit to get explained variance, if n_components is not directly specified)
    # If n_components is already specified, we'll directly use it.
    if n_components is None:
        pca = PCA() # Fit without specifying components to analyze explained variance
        pca.fit(scaled_data)
        explained_variance_ratio = pca.explained_variance_ratio_
        cumulative_explained_variance = np.cumsum(explained_variance_ratio)

        print("\nExplained variance per principal component:\n", explained_variance_ratio)
        print("\nCumulative explained variance:\n", cumulative_explained_variance)

        # Suggest optimal number of components based on min_variance_retention
        if len(cumulative_explained_variance) > 0:
            suggested_n_components = np.where(cumulative_explained_variance >= min_variance_retention)[0][0] + 1
            print(f"\nTo retain at least {min_variance_retention*100:.0f}% of the variance, "
                  f"it's suggested to select {suggested_n_components} principal components.")
            final_n_components = suggested_n_components
        else:
            final_n_components = 0
            print("No principal components could be determined from the data.")
    else:
        # If n_components is provided, use it directly
        final_n_components = n_components
        print(f"\nProceeding with PCA using n_components = {final_n_components}")

    # 4. Re-execute PCA with the determined or specified number of components
    if final_n_components > 0:
        pca_model = PCA(n_components=final_n_components)
        principal_components = pca_model.fit_transform(scaled_data)

        # Convert principal components to a DataFrame for easier handling
        pc_column_names = [f'PC{i+1}' for i in range(principal_components.shape[1])]
        pca_transformed_df = pd.DataFrame(data=principal_components, index=df.index, columns=pc_column_names)
        print(f"\nFinal PCA transformed data shape: {pca_transformed_df.shape}")
        print("PCA transformed data (first 5 rows):\n", pca_transformed_df.head())
    else:
        pca_transformed_df = pd.DataFrame(index=df.index) # Empty DataFrame if no components
        pca_model = None
        print("PCA could not be performed with the specified parameters or data.")
        
    # 5. Normolize results
    min_max_scaler = MinMaxScaler()
    # Apply Min-Max Scaling to the PCA transformed DataFrame
    normalized_pca_data = min_max_scaler.fit_transform(pca_transformed_df)
    normalized_pca_df = pd.DataFrame(normalized_pca_data,
                                     index=pca_transformed_df.index,
                                     columns=pca_transformed_df.columns)
    
    # 6. get loadings
    loadings_df = pd.DataFrame(pca_model.components_,
                               columns=original_feature_names,
                               index=pc_column_names)
    
    # 8. Visualization (Optional)
    if visualize and pca_model is not None:
        # Plot Explained Variance Ratio (if full PCA was done initially)
        if n_components is None and len(cumulative_explained_variance) > 0:
            plt.figure(figsize=(10, 6))
            plt.plot(range(1, len(explained_variance_ratio) + 1), cumulative_explained_variance, marker='o', linestyle='--')
            plt.axhline(y=min_variance_retention, color='r', linestyle=':', label=f'{min_variance_retention*100:.0f}% Variance')
            plt.axvline(x=final_n_components, color='g', linestyle=':', label=f'{final_n_components} Components')
            plt.title('PCA - Cumulative Explained Variance')
            plt.xlabel('Number of Principal Components')
            plt.ylabel('Cumulative Explained Variance Ratio')
            plt.xticks(range(1, len(explained_variance_ratio) + 1))
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            plt.show()

        # Scatter plot of the first two principal components (if at least 2 components are kept)
        # if final_n_components >= 2:
        #     plt.figure(figsize=(8, 6))
        #     sns.scatterplot(x=pca_transformed_df.iloc[:, 0], y=pca_transformed_df.iloc[:, 1],
        #                     hue=pca_transformed_df.index, s=100, palette='viridis')
        #     plt.title('PCA - Scatter Plot of First Two Principal Components')
        #     plt.xlabel(f'Principal Component 1 (explains {pca_model.explained_variance_ratio_[0]*100:.2f}%)')
        #     plt.ylabel(f'Principal Component 2 (explains {pca_model.explained_variance_ratio_[1]*100:.2f}%)')
        #     plt.grid(True)
        #     # plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        #     plt.tight_layout()
        #     plt.show()
        # elif final_n_components == 1:
        #     print("\nOnly one principal component was retained. A 2D scatter plot is not applicable.")
        # --- New: Plotting Feature Loadings (Contributions) ---
        
        print("\n--- Visualizing Feature Loadings ---")
        # pca_model.components_ gives the principal axes in feature space (loadings)
        # It has shape (n_components, n_features)

        plt.figure(figsize=(12, 8))
        sns.heatmap(loadings_df, cmap='coolwarm', annot=True, fmt=".2f", linewidths=.5, linecolor='black')
        plt.title('Principal Component Loadings (Contribution of Original Features to PCs)')
        plt.xlabel('Original Features')
        plt.ylabel('Principal Components')
        plt.tight_layout()
        plt.show()

        # Optional: Bar plot for individual PC loadings (e.g., PC1)
        if final_n_components >= 1:
            for pc in ['PC1', 'PC2', 'PC3', 'PC4', 'PC5','PC6']:
                plt.figure(figsize=(12, 6))
                loadings_df.loc[pc].plot(kind='bar')
                plt.title(f'Loadings of Original Features on Principal Component 1 {pc}')
                plt.xlabel('Original Features')
                plt.ylabel('Loading Value')
                plt.xticks(rotation=45, ha='right')
                plt.grid(axis='y', linestyle='--')
                plt.tight_layout()
                plt.show()
            
        print("\n--- Visualizing Clustered Feature Correlations ---")
        # Calculate the correlation matrix of the original (scaled) features
        plt.figure(figsize=(10, 8))
        # Use sns.clustermap which performs hierarchical clustering by default
        # The 'cbar_kws' argument helps to control the color bar label
        sns.clustermap(loadings_df, cmap='coolwarm', annot=True, fmt=".2f",
                       linewidths=.5, figsize=(12, 10),
                       cbar_kws={'label': 'Correlation Coefficient'})
        plt.suptitle('Clustered Heatmap of Original Feature Correlations', y=1.02) # Adjusted title position
        plt.show()
            
    print("\n--- PCA Analysis Complete ---")
    return normalized_pca_df, pca_model, loadings_df

## 1. Phonemic feature matrix
### 1.1. Get MFA dictionary

In [ ]:
def read_syllable_dict(file_path):
    """
    Reads a file and returns a dictionary with words as keys and phoneme lists as values.
    Ignores the numeric middle section between tabs, extracting only word and phonemes.
    
    Args:
        file_path (str): Path to the file
    
    Returns:
        dict: {word: [phonemes]}
    """
    syllable_dict = {}
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # Remove leading/trailing whitespace and split by tabs
            parts = line.strip().split('\t')

            # Extract word (first part) and phonemes (third part)
            word = parts[0].replace("'", "")  # Remove single quotes if present
            if len(parts) == 6:
                phonemes_str = parts[5]  # Third part is the phonemes
            elif len(parts) == 2:
                phonemes_str = parts[1]  # Third part is the phonemes
            else:
                continue

            # Split phonemes by spaces into a list
            phonemes = phonemes_str.split()
            if not phonemes:  # If no phonemes found, skip the line
                print(f"No phonemes found in line: {line.strip()}")
                continue
                
            # Store in dictionary
            syllable_dict[word] = phonemes
    
    return syllable_dict

In [ ]:
file_path = "english_us_lr.dict"
# Create the filename-to-phoneme dictionary
dict = read_syllable_dict(file_path)
for word, syllables in list(dict.items())[:10]:
    print(f"{word}: {syllables}")

### 1.2. Get the stimuli phonemic dictionary for the Lexical Delay Experiment

In [ ]:
import os

def get_wav_files(directory):
    wav_files_names = {}
    wav_files = []
    # Check if directory exists
    if not os.path.exists(directory):
        print(f"Directory '{directory}' does not exist.")
        return wav_files
    
    # Iterate through files in the directory
    for filename in os.listdir(directory):
        if filename.lower().endswith('.wav'):  # Case-insensitive check for .wav files
            # Add to dictionary (key is filename, value can be None or customized)
            wav_files_names[filename[:-4]] = None
    return wav_files_names

In [ ]:
def create_phoneme_start_time_dict(syllables, base_path:str='C:\\Users\\bl314\\Box\\CoganLab\\ECoG_Task_Data\\Stim\\Lexical\\mfa\\stim_annotations'):
    phoneme_start_times = {}
    for word in syllables.keys():
        print(f'getting timestamps of word {word}')
        file_path = os.path.join(base_path, f"{word}_phones.txt")
        try:
            with open(file_path, 'r') as f:
                start_times = []
                for i, line in enumerate(f):
                    if i < 5: 
                        parts = line.strip().split('\t')
                        if parts:
                            start_times.append(float(parts[0]))
                    else:
                        break
                phoneme_start_times[word] = start_times
        except FileNotFoundError:
            print(f"Not found {file_path}. Skipped {word}")
    return phoneme_start_times

In [ ]:
directory = r"C:\Users\bl314\Box\CoganLab\task_stimuli\LexicalDecRepDelay"
syllables = get_wav_files(directory)
for syllable in syllables.keys():
    syllables[syllable] = dict.get(syllable)
    if syllable=='pilot' and len(syllables[syllable])<5:
        syllables[syllable].append('T')
    print(f"{syllable}: {syllables[syllable]}")
syllables_timestamps = create_phoneme_start_time_dict(syllables)
syllables_timestamps_DF=pd.DataFrame(syllables_timestamps)
print(syllables_timestamps_DF)
# Get time of the first phoneme
print(np.mean(syllables_timestamps_DF.iloc[1]))
print(np.std(syllables_timestamps_DF.iloc[1]))

print(np.mean(syllables_timestamps_DF.iloc[2]))
print(np.std(syllables_timestamps_DF.iloc[2]))


In [ ]:
#For LME: save phoneme categories and timestamps for each syllable
import pickle
filename = os.path.join(lme_dir,'stim_pho_dict.pkl')
with open(filename, 'wb') as f:
    pickle.dump(syllables, f)
filename = os.path.join(lme_dir,'stim_pho_t_dict.pkl')
with open(filename, 'wb') as f:
    pickle.dump(syllables_timestamps, f)


### 1.3. One-hot encoding for each unique phoneme

In [ ]:
# Get unique phoneme vector
all_phonemes = set()
for phonemes in syllables.values():
    all_phonemes.update(phonemes)
all_phonemes = sorted(list(all_phonemes))  # Sort for consistent ordering
phoneme_to_index = {phoneme: idx for idx, phoneme in enumerate(all_phonemes)}
vector_length = len(all_phonemes)

# Create one-hot encoding for each word
phoneme_one_hot_dict = {}
for word, phonemes in syllables.items():
    # Initialize vector with zeros
    vector = [0] * vector_length
    # Set 1 for each phoneme present in the word
    for phoneme in phonemes:
        vector[phoneme_to_index[phoneme]] = 1
    phoneme_one_hot_dict[word] = vector

In [ ]:
# Check the multicolinearity and reduce the feature dimensions
check_multicollinearity(np.array(list(phoneme_one_hot_dict.values())),all_phonemes)
# Print the words tha contain AE0, W, ER0, and Y
for syllable, value in syllables.items():
    if value and "AE0" in value:
        print(f"{syllable}: {value} contains AE0")
    if value and "W" in value:
        print(f"{syllable}: {value} contains W")
    if value and "ER0" in value:
        print(f"{syllable}: {value} contains ER0")
    if value and "Y" in value:
        print(f"{syllable}: {value} contains Y")
    if value and "UW1" in value:
        print(f"{syllable}: {value} contains UW1")
    if value and "HH" in value and "M" in value:
        print(f"{syllable}: {value} contains both HH and M")
    if value and "T" in value and "L" in value and "AH0" in value:
        print(f"{syllable}: {value} contains T, L and AH0")
    if value and "OW1" in value and "M" in value and "N" in value:
        print(f"{syllable}: {value} contains OW1, M and N")
# I think it is safe to reduce ER0, Y, UW1, AE0, and W
# Then: humor - HH and M, tulip - 'T', 'L', 'AH0', 'P', women - 'OW1', 'M', 'N'

# reduce the dimensions:
indices_to_remove=[all_phonemes.index("ER0"),all_phonemes.index("Y"),all_phonemes.index("UW1"),all_phonemes.index("AE0"),all_phonemes.index("W")]
phoneme_one_hot_dict_filt = {key: vec for key, vec in phoneme_one_hot_dict.items()}
for i, (key, vec) in enumerate(phoneme_one_hot_dict.items()):
    # Create a copy of the vector with specified indices removed
    filtered_vec = np.delete(vec, indices_to_remove)
    phoneme_one_hot_dict_filt[key] = filtered_vec
all_phonemes_filt = []
all_phonemes_filt = [p for i, p in enumerate(all_phonemes) if i not in indices_to_remove]

# Check the multicolinearity again
check_multicollinearity(np.array(list(phoneme_one_hot_dict_filt.values())),all_phonemes_filt)

In [ ]:
# PCA
print("\n--- Running PCA with automatic component selection (95% variance) ---")
pca_results_auto, _, _ = perform_pca_on_acoustic_features(
    phoneme_one_hot_dict_filt,original_feature_names=all_phonemes_filt,
    n_components=None, # Let the function decide
    min_variance_retention=0.9
)
phoneme_one_hot_dict_filt_pca = pca_results_auto.to_dict('index')
phoneme_one_hot_dict_filt_pca = {key: list(value.values()) for key, value in phoneme_one_hot_dict_filt_pca.items()}

for key, values in list(phoneme_one_hot_dict_filt_pca.items())[:5]:  # Show first 5 entries
    print(key, ":", values)


In [ ]:
# save the phoneme_one_hot_dicphoneme_one_hot_dictt
import pickle
phoneme_one_hot_dict_path = "phoneme_one_hot_dict.pickle"
with open(phoneme_one_hot_dict_path, 'wb') as handle:
    pickle.dump(phoneme_one_hot_dict_filt, handle)
phoneme_one_hot_dict_filt

In [ ]:
import pandas as pd
phoneme_one_hot_dict_filt_df = pd.DataFrame(phoneme_one_hot_dict_filt)
phoneme_one_hot_dict_filt_df.to_csv(os.path.join(lme_dir,'phoneme_one_hot_dict.csv'), index=False)
phoneme_one_hot_dict_filt_pca_df = pd.DataFrame(phoneme_one_hot_dict_filt_pca)
phoneme_one_hot_dict_filt_pca_df.to_csv(os.path.join(lme_dir,'phoneme_one_hot_dict_pca.csv'), index=False)

In [ ]:
# # Read the saved one hot dict. Just for demonstration
# import pandas as pd
# d= pd.read_pickle("phoneme_one_hot_dict.pickle")

## 2. Acoustic feature (Power envelope at five frequency bins)
### 2.1. Read normolized binned envelope 
(The matrix was generated by `get_stims_envelope.m`)

In [ ]:
envelope_feature_dict = {}

with open("envelope_power_bins.txt", "r") as file:
    for line in file:
        parts = line.strip().split("\t")
        key = parts[0]
        values = list(map(float, parts[1:]))  # Convert remaining columns to float
        envelope_feature_dict[key] = values
feature_names=['50.00 Hz','120.58 Hz','209.04 Hz','319.88 Hz','458.78 Hz','632.84 Hz','850.96 Hz','1124.30 Hz','1466.84 Hz','1896.08 Hz','2433.98 Hz','3108.04 Hz','3952.74 Hz','5011.26 Hz','6337.74 Hz','8000.00 Hz']
# Print a sample of the dictionary
for key, values in list(envelope_feature_dict.items())[:5]:  # Show first 5 entries
    print(key, ":", values)
import pandas as pd

In [ ]:
envelope_feature_dict_df = pd.DataFrame(envelope_feature_dict)
envelope_feature_dict_df.to_csv(os.path.join(lme_dir,'envelope_feature_dict.csv'), index=False)

In [ ]:
# # Check the multicolinearity and reduce the feature dimensions
check_multicollinearity(np.array(list(envelope_feature_dict.values())),
                        feature_names)

In [ ]:
# PCA
print("\n--- Running PCA with automatic component selection (95% variance) ---")
pca_results_auto, _, _ = perform_pca_on_acoustic_features(
    envelope_feature_dict,original_feature_names=feature_names,
    n_components=None, # Let the function decide
    min_variance_retention=0.9
)
envelope_feature_dict_pca = pca_results_auto.to_dict('index')
envelope_feature_dict_pca = {key: list(value.values()) for key, value in envelope_feature_dict_pca.items()}

for key, values in list(envelope_feature_dict_pca.items())[:5]:  # Show first 5 entries
    print(key, ":", values)

In [ ]:
check_multicollinearity(np.array(list(envelope_feature_dict_pca.values())),
                        ['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9'])

In [ ]:
# Save dictionary
import pickle
envelope_feature_dict_path = "envelope_feature_dict.pickle"
with open(envelope_feature_dict_path, 'wb') as handle:
    pickle.dump(envelope_feature_dict_pca, handle)

In [ ]:
envelope_feature_dict_pca = pd.DataFrame(envelope_feature_dict_pca)
envelope_feature_dict_pca.to_csv(os.path.join(lme_dir,'envelope_feature_dict_pca.csv'), index=False)

## 3. Acoustic+Phonemeic feature (for a whole token)


In [ ]:
def classify_pca(loadings_df, ratio_threshold=6/4):
    hz_cols = [col for col in loadings_df.columns if 'Hz' in col]
    other_cols = [col for col in loadings_df.columns if 'Hz' not in col]
    
    if len(hz_cols) == 0 or len(other_cols) == 0:
        raise ValueError("Loadings DataFrame must contain at least one 'Hz' feature column and one non-'Hz' feature column.")

    pc_new_names = {}
    
    aco_counter = 1
    pho_counter = 1
    acpho_counter = 1

    for pc_idx, row in loadings_df.iterrows():
        aco_w = np.mean(np.abs(row[hz_cols]))
        pho_w = np.mean(np.abs(row[other_cols]))

        if pho_w / aco_w > ratio_threshold:
            new_name = f'pho{pho_counter}'
            pho_counter += 1
        elif aco_w / pho_w > ratio_threshold:
            new_name = f'aco{aco_counter}'
            aco_counter += 1
        else:
            new_name = f'mix{acpho_counter}'
            acpho_counter += 1
            
        pc_new_names[pc_idx] = new_name
    
    return pc_new_names

In [ ]:
# Load dictionary of HG responses and make a lexical status mapping
from ieeg.arrays.label import LabeledArray
script_dir = 'D:\\bsliang_Coganlabcode\\coganlab_ieeg\\projects\\lme\\data'
epoc_LexDelayRep_Aud = LabeledArray.fromfile(os.path.join(script_dir,'epoc_LexDelayRep_Aud'))
stim_lexsts = epoc_LexDelayRep_Aud.labels[0].split('/')[:,[3,2]]
del epoc_LexDelayRep_Aud
lexsts_dict = {str(item[0]): str(item[1]) for item in stim_lexsts}
lexsts_bi_dict = {key: [1] if value == 'Word' else [0] for key, value in lexsts_dict.items()}
lexsts_bi_dict['magic']=[1]
print(lexsts_bi_dict)

In [ ]:
aco_pho_lex_dict = {key: lexsts_bi_dict[key] + envelope_feature_dict[key] + phoneme_one_hot_dict_filt[key].tolist()  for key in envelope_feature_dict}
check_multicollinearity(np.array(list(aco_pho_lex_dict.values())),
                        ['lex']+feature_names+all_phonemes_filt)

In [ ]:
pca_results_aco_pho, _, loadings_df = perform_pca_on_acoustic_features(
    aco_pho_lex_dict,original_feature_names=['lex']+feature_names+all_phonemes_filt,
    n_components=None, # Let the function decide
    min_variance_retention=0.9
)
aco_pho_dict_pca = pca_results_aco_pho.to_dict('index')
aco_pho_dict_dict_pca = {key: list(value.values()) for key, value in aco_pho_dict_pca.items()}

In [ ]:
aco_pho_dict_pca_df = pd.DataFrame(aco_pho_dict_pca)
print(aco_pho_dict_pca_df)
pc_new_names=classify_pca(loadings_df,ratio_threshold=6/4)
aco_pho_dict_pca_df.rename(index=pc_new_names, inplace=True)
print(aco_pho_dict_pca_df)
sns.heatmap(loadings_df.rename(index=pc_new_names))
print(loadings_df.T)
aco_pho_dict_pca_df.to_csv(os.path.join(lme_dir,'aco_pho_dict_pca.csv'), index=True)

## 4. Acoustic+Phonemeic feature (pho1 or 2 only)


In [ ]:
pho=2

In [ ]:
# Get acoustic features
phoX_aco_dict = {}
with open(f"envelope_power_bins_pho{pho}.txt", "r") as file:
    for line in file:
        parts = line.strip().split("\t")
        key = parts[0]
        values = list(map(float, parts[1:]))  # Convert remaining columns to float
        phoX_aco_dict[key] = values
feature_names=['50.00 Hz','120.58 Hz','209.04 Hz','319.88 Hz','458.78 Hz','632.84 Hz','850.96 Hz','1124.30 Hz','1466.84 Hz','1896.08 Hz','2433.98 Hz','3108.04 Hz','3952.74 Hz','5011.26 Hz','6337.74 Hz','8000.00 Hz']
# Print a sample of the dictionary
for key, values in list(phoX_aco_dict.items())[:5]:  # Show first 5 entries
    print(key, ":", values)
check_multicollinearity(np.array(list(phoX_aco_dict.values())),
                        feature_names)

In [ ]:
# Get phoneme vector for phoX
all_phonemes_phoX = set()
for phonemes in syllables.values():
    all_phonemes_phoX.add(phonemes[pho-1])
print(all_phonemes_phoX)
all_phonemes_phoX = sorted(list(all_phonemes_phoX))  # Sort for consistent ordering
phoneme_to_index = {phoneme: idx for idx, phoneme in enumerate(all_phonemes_phoX)}
vector_length = len(all_phonemes_phoX)

# Create one-hot encoding for each word (only pho1)
phoX_pho_dict = {}
for word, phonemes in syllables.items():
    # Initialize vector with zeros
    vector = [0] * vector_length
    # Set 1 for each phoneme present in the word
    vector[phoneme_to_index[phonemes[pho-1]]] = 1
    phoX_pho_dict[word] = vector

check_multicollinearity(np.array(list(phoX_pho_dict.values())),
                        all_phonemes_phoX)

In [ ]:
# Combine the two and run PCA
phoX_aco_pho_dict = {key: phoX_aco_dict[key] + phoX_pho_dict[key]
                    for key in phoX_aco_dict}
check_multicollinearity(np.array(list(phoX_aco_pho_dict.values())),
                        feature_names+all_phonemes_phoX)
phoX_aco_pho_dict_pca_df, _, loadings_df = perform_pca_on_acoustic_features(
    phoX_aco_pho_dict,original_feature_names=feature_names+all_phonemes_phoX,
    n_components=None, # Let the function decide
    min_variance_retention=0.9
)
print(loadings_df)
phoX_aco_pho_dict_pca_df=phoX_aco_pho_dict_pca_df.T
print(phoX_aco_pho_dict_pca_df)
pc_new_names=classify_pca(loadings_df,ratio_threshold=6/4)
phoX_aco_pho_dict_pca_df.rename(index=pc_new_names, inplace=True)
print(phoX_aco_pho_dict_pca_df)
loadings_df=np.abs(loadings_df)
sns.heatmap(loadings_df.rename(index=pc_new_names))
phoX_aco_pho_dict_pca_df.to_csv(os.path.join(lme_dir,f'pho{pho}_aco_pho_dict_pca.csv'), index=True)


## 5. Acoustic+Phonemeic feature (syllable1 only)


In [ ]:
# Get acoustic features
syl1_aco_dict = {}
with open("envelope_power_bins_syl1.txt", "r") as file:
    for line in file:
        parts = line.strip().split("\t")
        key = parts[0]
        values = list(map(float, parts[1:]))  # Convert remaining columns to float
        syl1_aco_dict[key] = values
feature_names=['50.00 Hz','120.58 Hz','209.04 Hz','319.88 Hz','458.78 Hz','632.84 Hz','850.96 Hz','1124.30 Hz','1466.84 Hz','1896.08 Hz','2433.98 Hz','3108.04 Hz','3952.74 Hz','5011.26 Hz','6337.74 Hz','8000.00 Hz']
# Print a sample of the dictionary
for key, values in list(syl1_aco_dict.items())[:5]:  # Show first 5 entries
    print(key, ":", values)
check_multicollinearity(np.array(list(syl1_aco_dict.values())),
                        feature_names)

In [ ]:
# Get phoneme vector for pho1
all_phonemes_syl1 = set()
for phonemes in syllables.values():
    all_phonemes_syl1.add(phonemes[0])
    all_phonemes_syl1.add(phonemes[1])
all_phonemes_syl1 = sorted(list(all_phonemes_syl1))  # Sort for consistent ordering
phoneme_to_index = {phoneme: idx for idx, phoneme in enumerate(all_phonemes_syl1)}
vector_length = len(all_phonemes_syl1)

# Create one-hot encoding for each word (only pho1)
syl1_pho_dict = {}
for word, phonemes in syllables.items():
    # Initialize vector with zeros
    vector = [0] * vector_length
    # Set 1 for each phoneme present in the word
    vector[phoneme_to_index[phonemes[0]]] = 1
    vector[phoneme_to_index[phonemes[1]]] = 1
    syl1_pho_dict[word] = vector

check_multicollinearity(np.array(list(syl1_pho_dict.values())),
                        all_phonemes_syl1)

In [ ]:
# Combine the two and run PCA
syl1_aco_pho_dict = {key: syl1_aco_dict[key] + syl1_pho_dict[key]
                    for key in syl1_aco_dict}
check_multicollinearity(np.array(list(syl1_aco_pho_dict.values())),
                        feature_names+all_phonemes_syl1)
syl1_aco_pho_dict_pca_df, _, loadings_df = perform_pca_on_acoustic_features(
    syl1_aco_pho_dict,original_feature_names=feature_names+all_phonemes_syl1,
    n_components=None, # Let the function decide
    min_variance_retention=0.9
)
print(loadings_df)
syl1_aco_pho_dict_pca_df=syl1_aco_pho_dict_pca_df.T
print(syl1_aco_pho_dict_pca_df)
pc_new_names=classify_pca(loadings_df,ratio_threshold=6/4)
syl1_aco_pho_dict_pca_df.rename(index=pc_new_names, inplace=True)
print(syl1_aco_pho_dict_pca_df)
loadings_df=np.abs(loadings_df)
sns.heatmap(loadings_df.rename(index=pc_new_names))
syl1_aco_pho_dict_pca_df.to_csv(os.path.join(lme_dir,'syl1_aco_pho_dict_pca.csv'), index=True)